# ClassyMIDI (Ver 1.5)

## Simple, yet powerful MIDI classifier and songs names generator.

***

### Guo R, Simpson I, Magnusson T, Kiefer C., Herremans D.. 2020. A variational autoencoder for music generation controlled by tonal tension. Joint Conference on AI Music Creativity (CSMC + MuMe).

https://github.com/ruiguo-bio/midi-miner

***

#### Project Los Angeles

#### Tegridy Code 2021

***

# Setup Environment

In [ ]:
#@title Install dependencies
!git clone https://github.com/asigalov61/tegridy-tools
!pip install pypianoroll
!pip install texttable
!pip install fuzzywuzzy[speedup]
!pip install tqdm
!pip install pretty_midi

In [ ]:
#@title Import needed modules and create IO dirs

print('Loading all needed modules. Please wait...')

import secrets
import sys
import os

os.chdir('/content/tegridy-tools/tegridy-tools')
import TMIDI
import ClassyMIDI
os.chdir('/content/')

import tqdm
from tqdm import auto

from shutil import copyfile

import pypianoroll
from pypianoroll import Multitrack

from texttable import Texttable

from pprint import pprint

import zipfile

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

from glob import glob

import pretty_midi


import numpy as np

import pickle

import pandas as pd

import scipy.stats

from collections import Counter

from functools import reduce

from copy import deepcopy

import json

import logging

print('Creating IO dirs...')

# Make IO dirs
os.chdir('/content')
print('Prepping IO dirs...')
if not os.path.exists('/content/Songs'):
  os.makedirs('/content/Songs')
if not os.path.exists('/content/Output'):
  os.makedirs('/content/Output')
if not os.path.exists('/content/Input'):
  os.makedirs('/content/Input')

print('Done! Enjoy! :)')

# Single MIDI classification metrics

In [ ]:
#@title Print music metrics for a MIDI file
full_path_to_MIDI_file = "/content/tegridy-tools/tegridy-tools/seed2.mid" #@param {type:"string"}

PR, SIG = ClassyMIDI.get_music_metrics(full_path_to_MIDI_file, 4, 0)
X = ClassyMIDI.all_features(full_path_to_MIDI_file)

print('Printing all features...')
print('=' * 100)

print('track_programs:', X[0][0])
print('track_names:', X[0][1])
print('file_names:', X[0][2])
print('is_drum:', X[0][3])
print('=' * 100)

print('Duration:', X[0][4])
print('Number of notes:', X[0][5])
print('Occupation rate:', X[0][6])
print('Polyphony rate:', X[0][7])
print('=' * 100)

print('Highest Pitch:', X[0][8])
print('Lowest Pitch:', X[0][9])
print('Pitch Avg:', X[0][10])
print('Pitch Std Dev:', X[0][11])
print('=' * 100)

print('Highest pitch norm', X[0][12])
print('lowest pitch norm:', X[0][13])
print('pitch mode norm:', X[0][14])
print('pitch std norm:', X[0][15])
print('=' * 100)

print('number of interval:', X[0][16])
print('largest interval:', X[0][17])
print('smallest interval:', X[0][18])
print('interval mode:', X[0][19])
print('=' * 100)

print('number of interval norm:', X[0][20])
print('largest interval norm:', X[0][21])
print('smallest interval norm:', X[0][22])
print('interval mode norm:', X[0][23])
print('longest note duration:', X[0][24])
print('shortest note duration:', X[0][25])
print('=' * 100)

print('mean note duration:', X[0][26])
print('longest note duration:', X[0][27])
print('note duration std:', X[0][28])
print('longest note duration norm:', X[0][29])
print('shortest note duration norm:', X[0][30])
print('mean note duration norm:', X[0][31])
print('note duration std norm:', X[0][32])
print('=' * 100)

SIG_SIMPLE = [SIG[0], SIG[1] * 1, SIG[2], SIG[3]]
SIG_FULL = [float(X[0][4] * 1), # Duration
            float(X[0][5]),
            float(X[0][6] * 1),
            float(X[0][7] * 1),
            
            float(X[0][8]), # Highest Pitch
            float(X[0][9]),
            float(X[0][10]),
            float(X[0][11]),

            float(X[0][12] * 1), # Highest pitch norm
            float(X[0][13] * 1),
            float(X[0][14] * 1),
            float(X[0][15] * 1),

            float(X[0][16]), # number of interval
            float(X[0][17]),
            float(X[0][18]),
            float(X[0][19]),


            float(X[0][20] * 1), # number of interval norm
            float(X[0][21] * 1),
            float(X[0][22] * 1),
            float(X[0][23] * 1),
            float(X[0][24] * 1),
            float(X[0][25] * 1),

            float(X[0][26] * 1), # mean note duration
            float(X[0][27] * 1),
            float(X[0][28] * 1),
            float(X[0][29] * 1),
            float(X[0][30] * 1),
            float(X[0][31] * 1),
            float(X[0][32] * 1)]

print('Resulting simple signature:', SIG_SIMPLE)
print('Resulting simple signature length:', len(SIG_SIMPLE))
print('Resulting simple signature summ:', sum(SIG_SIMPLE))
print('=' * 100)

print('Resulting simple signature:', SIG_FULL)
print('Resulting simple signature length:', len(SIG_FULL))
print('Resulting simple signature summ:', sum(SIG_FULL))
print('=' * 100)

# MIDI classification and songs names generation

# Download a sample MIDI dataset for training the classifier

In [ ]:
#@title Download and unzip clean_midi MIDI dataset
os.chdir('/content/Input')
!wget http://hog.ee.columbia.edu/craffel/lmd/clean_midi.tar.gz
!tar -xvf /content/Input/clean_midi.tar.gz

In [ ]:
#@title Process your dataset in Input dir

#@markdown NOTE: Processing large dataset may take a while so keep it in mind.

#@markdown NOTE: NOTE-Class is not implemented in bulk classification code below. If you need NOTE-Class you will have to do it yourself.

#@markdown NOTE: You can enable NOTE_Class below (very slow, bulky)
enable_NOTE_Class = False #@param {type:"boolean"}


print('ClassyMIDI Classifier and Songs Names Generator')
print('=' * 100)

print('Loading MIDI files...')
print('This may take a while on a large dataset in particular.')

files_count = 0

dataset_addr = "/content/Input"
os.chdir(dataset_addr)
filez = [y for x in os.walk(dataset_addr) for y in glob(os.path.join(x[0], '*.mid'))]

CLASS_DATA = []
NOTE_DATA = []

print('Processing MIDI files. Please wait...')
for f in tqdm.auto.tqdm(filez):
  
  try:
    SIG_SIMPLE, SIG_FULL, ALL_F, PRL = ClassyMIDI.get_signatures(f)
    
    fn = os.path.basename(f)
    fnn = fn.split('.')[0]
    fnnn = []
    
    X = PRL.flatten()
    
    s = secrets.randbelow(len(X))
    
    Z = []

    if enable_NOTE_Class:

      Z = [TMIDI.Tegridy_Optimus_Sum_Intro_Rand_End_Sampler(f)]
      NOTE_DATA.append(Z)

    CLASS_DATA.append([f, fnn, fnnn, SIG_SIMPLE, SIG_FULL, ALL_F.tolist(), X[s:s+64].tolist(), Z ])  
  
    files_count += 1

  except KeyboardInterrupt:
    break

  except:
    print('Bad file', f)
    continue

In [ ]:
#@title Save the processed signatures to file
# Writing dataset to pickle file
TMIDI.Tegridy_Pickle_File_Writer(CLASS_DATA, '/content/ClassySigsDataset')

 # (Bulk) MIDI Classification and Songs Names Generation

 ## Choose ONE pre-processed dataset if you did not process yours above

 ### NOTE: Do not load pre-processed datasets if you loaded yours above

In [ ]:
#@title Load ClassyMIDI Signatures Pack (173467 MIDI signatures from the full LAKH dataset)

#@markdown NOTE: This is a very large dataset (700+ MB) so it needs at least 8GB of free RAM to load 

#@markdown NOTE: DO NOT RUN if you processed (and want to use) your own/sample dataset above
print('Downloading the dataset...Please wait...')
%cd /content/
!wget --no-check-certificate -O ClassySigs-LAKH-Full-Dataset.zip "https://onedrive.live.com/download?cid=8A0D502FC99C608F&resid=8A0D502FC99C608F%2118478&authkey=AAFp1jFdm99ANfw"

print('Unzipping the dataset...')
!unzip -j ClassySigs-LAKH-Full-Dataset.zip

print('Loading the dataset into memory...')
CLASS_DATA = TMIDI.Tegridy_Any_Pickle_File_Loader('/content/ClassySigs-LAKH-Full')

print('Loading complete! Enjoy! :)')

In [ ]:
#@title Load tegridy-tools ClassyMIDI Signatures Pack (clean_midi's 10359 MIDI signatures)

#@markdown NOTE: DO NOT RUN if you processed (and want to use) your own/sample dataset above
print('Loading the dataset...')
os.chdir('/content/tegridy-tools/tegridy-data')

!unzip ClassySigs_clean_midi_10359.zip

CLASS_DATA = TMIDI.Tegridy_Any_Pickle_File_Loader('/content/tegridy-tools/tegridy-data/ClassySigs_clean_midi_10359')

os.chdir('/content')
print('Done! Enjoy! :)')

In [ ]:
#@title Load tegridy-tools ClassyMIDI Signatures Pack (LAKH's 18000 MIDI signatures)

#@markdown NOTE: DO NOT RUN if you processed (and want to use) your own/sample dataset above

os.chdir('/content/')

with zipfile.ZipFile("/content/tegridy-tools/tegridy-data/ClassySigs_LAKH_18000.zip","r") as zip_ref:
    zip_ref.extractall("/content")
CLASS_DATA = TMIDI.Tegridy_Any_Pickle_File_Loader('/content/ClassySigs_LAKH_18000')
os.chdir('/content/')
print('Done! Enjoy! :)')

## Classify

In [ ]:
#@title Load and Classify a single MIDI
full_path_to_MIDI_file = "/content/tegridy-tools/tegridy-tools/seed2.mid" #@param {type:"string"}

print('ClassyMIDI Classifier and Songs Names Generator')
print('=' * 100)

MIDI_File = full_path_to_MIDI_file
SIG_S, SIG_F, ALL_F, PRL = ClassyMIDI.get_signatures(MIDI_File)
SUM, INTRO, RAND, END = TMIDI.Tegridy_Optimus_Sum_Intro_Rand_End_Sampler(full_path_to_MIDI_file)


print('Matching MIDI', MIDI_File)
print('=' * 100)

print('MIDI Simple Signature', SIG_S)
print('=' * 100)

print('MIDI Full Signature', SIG_F)
print('=' * 100)

RATINGS_LIST = []

for i in auto.tqdm(range(1, len(CLASS_DATA))):
  RATINGS_LIST.append(fuzz.ratio(SIG_S, CLASS_DATA[i][3]))

MAX_idx = max(RATINGS_LIST)  
MATCH_idx = RATINGS_LIST.index(MAX_idx)

print('Best simple match rating is', MAX_idx)
print('Best simple match is', CLASS_DATA[MATCH_idx])
print('=' * 100)

RATINGS_LIST = []

for i in auto.tqdm(range(1, len(CLASS_DATA))):
  RATINGS_LIST.append(fuzz.ratio(SIG_F, CLASS_DATA[i][4]))

MAX_idx = max(RATINGS_LIST)  
MATCH_idx = RATINGS_LIST.index(MAX_idx)

print('Best precise rating is', MAX_idx)
print('Best precise match is', CLASS_DATA[MATCH_idx])
print('=' * 100)

try:
  RATINGS_LIST = []

  for i in auto.tqdm(range(1, len(CLASS_DATA))):
    RATINGS_LIST.append(fuzz.ratio([SUM, INTRO], [CLASS_DATA[i][7][0][0], CLASS_DATA[i][7][0][1]]))

  MAX_idx = max(RATINGS_LIST)  
  MATCH_idx = RATINGS_LIST.index(MAX_idx)

  print('Best NOTE-Class rating is', MAX_idx)
  print('Best NOTE-Class match is', CLASS_DATA[MATCH_idx])
  print('=' * 100)

except:
  print('NOTE-Class data is missing from CLASS_DATA. Check code and re-run classifier processor if needed.')
  print('Skipping NOTE-Class classification...')
print('Done! Enjoy! :)')

In [ ]:
#@title Bulk classification and MIDI songs naming

#@markdown 1) UPLOAD YOUR FILES TO THE "Songs" dir before running this cell/code

#@markdown 2) NOTE: Copies of renamed source midis will be copied to the Output dir for reference and for convenience

#@markdown 3) Select desired matching criteria

#@markdown 4) full_float_signature matching is very slow/resource intensive. Make sure you have enough resources to run in this mode


match_by = "full_signature" #@param ["full_signature", "short_signature", "random_piano_roll_signature", "full_float_signature"]
leave_original_file_names_on_copies = False #@param {type:"boolean"}
desired_prefix_for_song_new_name = "" #@param {type:"string"}

print('Classy MIDI Classifier and Songs Names Generator')
print('=' * 100)

print('Classifying and naming MIDI files...')
print('This may take a while on a large dataset in particular.')
print('=' * 100)

print('Prepping IO dirs...')
if not os.path.exists('/content/Songs'):
  os.makedirs('/content/Songs')
if not os.path.exists('/content/Output'):
  os.makedirs('/content/Output')
if not os.path.exists('/content/Input'):
  os.makedirs('/content/Input')

print('Initialzing...')
files_count = 0
SONG_DATA = []
SONG_CLASS_DATA = []
SONGS_NAMES = []
FINAL_NAMES_LIST = []

print('Gathering file names...')
dataset_addr = "/content/Songs"
os.chdir(dataset_addr)
filez = [y for x in os.walk(dataset_addr) for y in glob(os.path.join(x[0], '*.mid'), recursive=True)]

print('Processing MIDI files. Please wait...')
print('=' * 100)
for f in tqdm.auto.tqdm(filez):
  try:
    SIG_SIMPLE, SIG_FULL, ALL_F, PRL = ClassyMIDI.get_signatures(f)
    fn = os.path.basename(f)
    fnn = fn.split('.')[0]
    fnnn = []

    X = list(PRL.flatten())
    s = secrets.randbelow(len(X))
    SONG_DATA.append([f, fnn, fnnn, SIG_SIMPLE, SIG_FULL, ALL_F.tolist(), X[s:s+64]])

    RATINGS_LIST = []
    song_name = ''

    for i in range(1, len(CLASS_DATA)):
      if match_by == 'full_signature':
        RATINGS_LIST.append(fuzz.ratio(SIG_FULL, CLASS_DATA[i][4]))
      
      if match_by == 'short_signature':
        RATINGS_LIST.append(fuzz.ratio(SIG_SIMPLE, CLASS_DATA[i][3]))
      
      if match_by == 'random_piano_roll_signature':
        RATINGS_LIST.append(fuzz.ratio(PRL.tolist(), CLASS_DATA[i][-1]))
      
      if match_by == 'full_float_signature':
        RATINGS_LIST.append(fuzz.ratio(ALL_F.tolist(), CLASS_DATA[i][-2]))
    
    MAX_idx = max(RATINGS_LIST)
    MATCH_idx = RATINGS_LIST.index(MAX_idx)

    SONG_CLASS_DATA.append(CLASS_DATA[MATCH_idx])
    
    if CLASS_DATA[MATCH_idx][2] is str:
      song_name = CLASS_DATA[MATCH_idx][2].replace('.MID', '').replace('.mid', '').replace('/', '_').replace(' ', '_').replace('.', '_') #.replace('-', '_')
    
    if len(CLASS_DATA[MATCH_idx][2]) > 1:
      song_name = CLASS_DATA[MATCH_idx][2][0].replace('.MID', '').replace('.mid', '').replace('/', '_').replace(' ', '_').replace('.', '_') #.replace('-', '_')
    else:
      song_name = CLASS_DATA[MATCH_idx][2][0].replace('.MID', '').replace('.mid', '').replace('/', '_').replace(' ', '_').replace('.', '_') #.replace('-', '_')
    
    if CLASS_DATA[MATCH_idx][2] == []:
      song_name = CLASS_DATA[MATCH_idx][1].replace('.MID', '').replace('.mid', '').replace('/', '_').replace(' ', '_').replace('.', '_') #.replace('-', '_')

    SONGS_NAMES.append(song_name)
    FINAL_NAMES_LIST.append([fnn, song_name, MAX_idx])
    
    if leave_original_file_names_on_copies:
      fnnn = '../Output/' + fnn + '---' + song_name + '_' + str(files_count) + '.mid'
    else:
      fnnn = '../Output/' + song_name + '_' + str(files_count) + '.mid'

    if desired_prefix_for_song_new_name != '' and leave_original_file_names_on_copies == False:
      fnnn = '../Output/' + desired_prefix_for_song_new_name + '---' + song_name + '_' + str(files_count) + '.mid'
      

    copyfile(f, fnnn)
    print('=' * 100)
    
    print(fnn, '|', MAX_idx, '|', song_name.replace('__', '-').replace('_', ' '), '|')
    print('=' * 100)
    
    pprint([f, fnn, fnnn, SIG_SIMPLE, SIG_FULL, ALL_F.tolist(), X[s:s+64]], compact=True)
    print('=' * 100)
    
    files_count += 1

  except KeyboardInterrupt:
    break

  except:
    print('Bad file', f)
    continue

print('Done! Enjoy! :)')

# Print results if needed

In [ ]:
#@title Print SONG_DATA

pprint(SONG_DATA, compact=True)

In [ ]:
#@title Print SONG_CLASS_DATA

pprint(SONG_CLASS_DATA, compact=True)

In [ ]:
#@title Print SONGS_NAMES

pprint(SONGS_NAMES, compact=True)

In [ ]:
#@title Print FINAL_NAMES_LIST

pprint(FINAL_NAMES_LIST, compact=True)

In [ ]:
#@title Save the above data as a pickle file
TMIDI.Tegridy_Pickle_File_Writer([SONG_DATA, SONG_CLASS_DATA, SONGS_NAMES, FINAL_NAMES_LIST], '/content/ClassyMIDI-Bulk-Conversion-Data')

# Congrats! You did it! :)